# Рекомендация тарифов для телекоммуникационной компании

**Задача:**

Необходимо построить модель для задачи классификации, которая сможет подобрать подходящий для пользователя тариф.

**Данные:**

Имеются данные о поведении клиентов, котороые уже перешли на тарифы телекоммуникационной компании.

> Предобработка данных не понадобится она уже сделана ([Исследование тарифов телекоммуникационной компании](https://github.com/SajKey/yandex_practicum_projects/tree/main/04_determination_of_a_profitable_plan_for_a_telecom_company)).

## Изучение общей информации данных

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('users_behavior.csv')

In [4]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Приведем тип данных в столбцах `calls` и `messages` к `int`, т.к. нет смысла оставлять их в типе `float`.

In [6]:
for column in df[['calls', 'messages']]:
    df[column] = df[column].astype('int')

In [7]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0


**Вывод:**

Данные чистые, без пропусков и подготовленны для дальнейшей работы с ними.

## Разбиение данных на выборки

In [8]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Спрятанной тестовой выборки нет. Разобъем данные на три части: обучающая, валидационная и тестовая. Размеры тестового и валидационного наборов равны. Исходные данные разбиваем в соотношении 3:1:1.

In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

Размер обучаемой выборки:

In [10]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


Разделение на валидационную и тестовую выборки.

In [11]:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345) 

Размер валидационной выборки:

In [12]:
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)


Размер тестовой выборки:

In [13]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


## Исследование моделей

Исследование лучшей модели **Decision Tree Classifier**

In [14]:
model = DecisionTreeClassifier(random_state=12345)
parametrs = {'max_depth':range(1,6), 
             'max_leaf_nodes':range(1,11)}
grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_train, target_train)
predictions_valid = grid.predict(features_valid) 
result = accuracy_score(target_valid, predictions_valid) 

print(grid.best_params_)
print(f'Лучшая оценка доли правильных ответов: {result}')

{'max_depth': 5, 'max_leaf_nodes': 7}
Лучшая оценка доли правильных ответов: 0.7869362363919129


Исследование лучшей модели **Random Forest Classifier**

In [15]:
model = RandomForestClassifier(random_state=12345)
parametrs = {'max_depth':range(5,11), 
             'n_estimators':range(11,31,10),
             'min_samples_leaf': range (1,3),
             'min_samples_split': range (2,7,2)}
grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_train, target_train)
predictions_valid = grid.predict(features_valid) 
result = accuracy_score(target_valid, predictions_valid) 

print(grid.best_params_)           
print(f'Лучшая оценка доли правильных ответов: {result}')

{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 11}
Лучшая оценка доли правильных ответов: 0.8009331259720062


Исследование модели **Logistic Regression**

In [16]:
model = LogisticRegression(random_state=12345)
parametrs = {'max_iter': [20, 50, 100, 200, 500, 1000],                      
             'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(features_train, target_train)
predictions_valid = grid.predict(features_valid) 
result = accuracy_score(target_valid, predictions_valid)

print(grid.best_params_) 
print(f'Лучшая оценка доли правильных ответов: {result}')

{'max_iter': 50, 'solver': 'newton-cg'}
Лучшая оценка доли правильных ответов: 0.7558320373250389


**Вывод:**

- Модель Decision Tree Classifier при параметре глубины дерева (`max_depth`) равному 5 и при максимальном количестве листьев в дереве (`max_leaf_nodes`) равному 7, выдает долю правильных ответов в 0.7869362363919129;
- Модель Random Forest Classifier при следующих параметрах: колличества деревьев(`n_estimators`) равным 11, глубины дерева (`max_depth`) равному 10, минимальным числом объектов на листе (`min_samples_leaf`) равным 2 и минимальным числом выборок, необходимых для разделения (`min_samples_split`) равным 6, выдает долю правильных ответов в 0.8009331259720062;
- Модель Logistic Regression при параметрах количества итераций (`max_iter`) равным 50 и алгоритма для оптимизации (`solver`) `newton-cg` выдает долю правильных ответов в 0.7573872472783826.

Из данного исследования можно сказать, что с данными лучше всего справилась модель **Random Forest Classifier**.

## Проверка модели на тестовой выборке

Проверка модели **Random Forest Classifier** на тестовой выборке.

In [17]:
model_test = RandomForestClassifier(random_state=12345, n_estimators=11, max_depth=10, min_samples_leaf=2, min_samples_split=6)
model_test.fit(features_train, target_train) 
predictions_test = model_test.predict(features_test)
result = accuracy_score(target_test, predictions_test) 
            
print(f'Доля правильных ответов: {result}')

Доля правильных ответов: 0.807153965785381


**Вывод:**

На тестовой выборке доля правильных ответов составила 0.807153965785381.

## Проверка модели на адекватность

In [18]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print(result_dc)

0.7060653188180405


In [19]:
print(f'Доля правильных ответов: {result}')

Доля правильных ответов: 0.807153965785381


**Вывод:**

Доля правильных ответов модели DummyClassifier ниже, чем у модели RandomForestClassifier, значит модель адекватна и эффективна.

## Общий вывод

Было проведено исследования моделей, для выбора лучшей модели для задачи классификации, которая выберет подходящий тариф. По итогам исследования на обучающих и валидационных выборках, была выбрана модель **Random Forest Classifier** с лучшей долей правильных ответов. Также эта модель была протестирована и проверена на адекватность, показав долю правильных ответов в 0.807153965785381.